In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

# 2.16.1 Statistical functions

In [5]:
ser = pd.Series(np.random.randn(8))
ser.pct_change()

0         NaN
1   -1.923373
2   -2.931038
3   -0.935503
4   -0.123427
5    7.136023
6   -1.639340
7   -2.450775
dtype: float64

In [7]:
df = DataFrame(np.random.rand(10, 4))
df.pct_change(3)

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,-0.020591,12.475995,0.967325,-0.553468
4,1.522533,-0.759493,4.268283,-0.128637
5,-0.839266,4.969317,-0.877771,-0.342039
6,-0.230832,0.994627,-0.320743,1.306429
7,0.511824,3.978803,2.442230,-0.767423
8,1.038695,14.972710,17.444836,0.235789
9,-0.085083,0.054706,0.009563,-0.911786


In [14]:
s = pd.Series(range(1,6))
print(s)
s.pct_change(3)

0    1
1    2
2    3
3    4
4    5
dtype: int64


0    NaN
1    NaN
2    NaN
3    3.0
4    1.5
dtype: float64

In [16]:
s1 = pd.Series(range(1,11))
s2 = pd.Series(range(2,21,2))
s1.cov(s2)

18.333333333333332

In [18]:
frame = pd.DataFrame(np.random.randn(1000, 5), columns=["a", "b", "c", "d", "e"])
frame.cov()

,a,b,c,d,e
a,1.050728,0.021590,-0.033379,0.031233,0.035940
b,0.021590,1.030669,0.022360,-0.035406,0.055048
c,-0.033379,0.022360,1.059442,0.056878,-0.038169
d,0.031233,-0.035406,0.056878,0.933382,0.052404
e,0.035940,0.055048,-0.038169,0.052404,0.895523


In [20]:
frame = pd.DataFrame(np.random.randn(20, 3), columns=["a", "b", "c"])
frame.loc[frame.index[:5], "a"] = np.nan
frame.loc[frame.index[5:10], "b"] = np.nan
frame.cov()
frame.cov(min_periods=12)

,a,b,c
a,0.969831,NaN,0.318122
b,NaN,0.937038,-0.037230
c,0.318122,-0.037230,0.880216


In [24]:
s1 = pd.Series(range(1,6))
s2 = pd.Series(range(6,11))
s2[:3] = np.nan
s1.cov(s2, min_periods=2)

0.5

In [26]:
frame = pd.DataFrame(np.random.randn(20, 3), columns=["a", "b", "c"])
frame.loc[frame.index[:5], "a"] = np.nan
frame.loc[frame.index[5:10], "b"] = np.nan
frame.corr()
frame.corr(min_periods=12)

,a,b,c
a,1.000000,NaN,-0.268127
b,NaN,1.000000,-0.058104
c,-0.268127,-0.058104,1.000000


In [27]:
def histogram_intersection(a, b):
    return np.minimum(np.true_divide(a, a.sum()), 
                      np.true_divide(b, b.sum())).sum()

frame.corr(method=histogram_intersection)

,a,b,c
a,1.000000,-0.449048,-6.072194
b,-0.449048,1.000000,-0.541214
c,-6.072194,-0.541214,1.000000


In [28]:
index = ["a", "b", "c", "d", "e"]
columns = ["one", "two", "three", "four"]
df1 = pd.DataFrame(np.random.randn(5, 4), index=index, columns=columns)
df2 = pd.DataFrame(np.random.randn(4, 4), index=index[:4], columns=columns)

df1.corrwith(df2)

one      0.517229
two     -0.014314
three   -0.362604
four    -0.563880
dtype: float64

In [30]:
s = pd.Series(np.random.randn(5), index=list("abcde"))
s["d"] = s["b"] # so there's a tie
s.rank()

a    1.688946
b    1.489742
c   -0.845329
d    1.489742
e    0.695434
dtype: float64

In [31]:
s.rank()

a    5.0
b    3.5
c    1.0
d    3.5
e    2.0
dtype: float64

In [36]:
df = pd.DataFrame(np.random.randn(10, 6))
df[4] = df[2][:5] # some ties
df.rank(1, ascending=False)

,0,1,2,3,4,5
0,6.0,1.0,2.5,4.0,2.5,5.0
1,6.0,3.0,1.5,4.0,1.5,5.0
2,5.0,4.0,2.5,1.0,2.5,6.0
3,3.0,1.0,4.5,2.0,4.5,6.0
4,2.0,4.0,5.5,1.0,5.5,3.0
5,3.0,1.0,4.0,5.0,NaN,2.0
6,4.0,1.0,3.0,5.0,NaN,2.0
7,3.0,1.0,4.0,2.0,NaN,5.0
8,5.0,4.0,3.0,2.0,NaN,1.0
9,4.0,2.0,3.0,1.0,NaN,5.0


# Group by

## 2.17.1 Splitting an object into groups

In [8]:
df = pd.DataFrame([
    ("bird", "Falconiformes", 389.0),
    ("bird", "Psittaciformes", 24.0),
    ("mammal", "Carnivora", 80.2),
    ("mammal", "Primates", np.nan),
    ("mammal", "Carnivora", 58),
],
   index=["falcon", "parrot", "lion", "monkey", "leopard"],
   columns=("class", "order", "max_speed"), 
)
df


# default is axis=0
grouped = df.groupby('class')
grouped = df.groupby('order', axis='columns')
grouped = df.groupby(["class", "order"])

In [9]:
df = pd.DataFrame({
    "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
    "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
    "C": np.random.randn(8),
    "D": np.random.randn(8),
})
df

,A,B,C,D
0,foo,one,-0.410294,0.897230
1,bar,one,-1.665443,0.618558
2,foo,two,1.287168,0.623793
3,bar,three,0.572997,-1.969315
4,foo,two,0.505494,2.211682
5,bar,two,0.808721,-0.100760
6,foo,one,1.955959,1.619905
7,foo,three,-1.102080,-2.951034


In [15]:
df2 = df.set_index(["A", "B"])
grouped = df2.groupby(df2.index.names.difference(["B"]))
grouped.sum()

,C,D
A,,
bar,-0.283725,-1.451516
foo,2.236245,2.401575


In [19]:
    def get_letter_type(letter):
        if letter.lower() in 'aeiou':
            return 'vowel'
        else:
            return 'consonant'
    df.groupby(get_letter_type, axis=1).apply(lambda x: x.values)

consonant    [[one, -0.4102942797204193, 0.8972297170043543...
vowel        [[foo], [bar], [foo], [bar], [foo], [bar], [fo...
dtype: object

In [24]:
lst = [1, 2, 3, 1, 2, 3]
s = pd.Series([1, 2, 3, 10, 20, 30], lst)
grouped = s.groupby(level=0)
grouped.first()
grouped.last()
grouped.sum()

1    11
2    22
3    33
dtype: int64

In [27]:
df2 = pd.DataFrame({"X": ["B", "B", "A", "A"], "Y": [1, 2, 3, 4]})
df2.groupby('X', sort=False).sum()
df2.groupby('X').sum()

,Y
X,
A,7
B,3


In [30]:
df3 = pd.DataFrame({"X": ["A", "B", "A", "B"], "Y": [1, 4, 3, 2]})
df3.groupby('X').get_group('A')
df3.groupby('X').get_group('B')

,X,Y
1,B,4
3,B,2


In [34]:
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dna = pd.DataFrame(df_list, columns=["a", "b", "c"])
df_dna

# Default ``dropna`` is set to True, which will exclude NaNs in keys
df_dna.groupby(by="b").sum()
df_dna.groupby(by="b", dropna=False).sum()

,a,c
b,,
1.0,2,3
2.0,2,5
NaN,1,4


In [41]:
df.groupby('A').groups

{'bar': [1, 3, 5], 'foo': [0, 2, 4, 6, 7]}

In [42]:
 df.groupby(get_letter_type, axis=1).groups

{'consonant': ['B', 'C', 'D'], 'vowel': ['A']}

In [46]:
df
grouped = df.groupby(['A','B'])
grouped.groups
len(grouped)

6

In [55]:
d="""date,height,weight,gender
2000-01-01,42.849980,157.500553,male
2000-01-02,49.607315,177.340407,male
2000-01-03,56.293531,171.524640,male
2000-01-04,48.421077,144.251986,female
2000-01-05,46.556882,152.526206,male
2000-01-06,68.448851,168.272968,female
2000-01-07,70.757698,136.431469,male
2000-01-08,58.909500,176.499753,female
2000-01-09,76.435631,174.094104,female
2000-01-10,45.306120,177.540920,male"""
df=g(d)
df['date'] = pd.to_datetime(df.date)
df = df.set_index('date')
gb = df.groupby('gender')
gb.<TAB>

In [59]:
arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
]
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
s = pd.Series(np.random.rand(8), index=index)
s

# We can then group by one of the levels in s.
grouped = s.groupby(level=1)
grouped.sum()

# If the MultiIndex has names specified, these can be passed instead of the level number:
s.groupby(level='second').sum()


second
one    0.820280
two    1.880629
dtype: float64

In [60]:
s.sum(level='second')

second
one    0.820280
two    1.880629
dtype: float64

In [66]:
arrays = [["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
          ["one", "two", "one", "two", "one", "two", "one", "two"],
         ]
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
df = pd.DataFrame({"A": [1, 1, 1, 1, 2, 2, 3, 3], "B": np.arange(8)},
                  index=index)
df.groupby(['A', pd.Grouper(level=1)]).sum()
df.groupby(['A', pd.Grouper(level='second')]).sum()


B
A second   
1 one     2
  two     4
2 one     4
  two     5
3 one     6
  two     7

In [69]:
df.groupby(['second','A',]).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [73]:
df = pd.DataFrame({
    "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
    "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
    "C": np.random.randn(8),
    "D": np.random.randn(8),
})
grouped = df.groupby(["A"])
grouped['C']
grouped.D

In [85]:
grouped = df.groupby('A')
for name, group in grouped:
    print(name)
    print(group)

bar
     A      B         C         D
1  bar    one  0.477848  0.671811
3  bar  three -0.005963  0.162831
5  bar    two -1.771557  0.554359
foo
     A      B         C         D
0  foo    one  1.187529 -1.074000
2  foo    two -0.348865  0.370527
4  foo    two -0.679904  0.135304
6  foo    one  1.609646 -0.824627
7  foo  three  0.815822  2.583978


In [87]:
for name, group in df.groupby(['A','B']):
    print(name)
    print(group)

('bar', 'one')
     A    B         C         D
1  bar  one  0.477848  0.671811
('bar', 'three')
     A      B         C         D
3  bar  three -0.005963  0.162831
('bar', 'two')
     A    B         C         D
5  bar  two -1.771557  0.554359
('foo', 'one')
     A    B         C         D
0  foo  one  1.187529 -1.074000
6  foo  one  1.609646 -0.824627
('foo', 'three')
     A      B         C         D
7  foo  three  0.815822  2.583978
('foo', 'two')
     A    B         C         D
2  foo  two -0.348865  0.370527
4  foo  two -0.679904  0.135304


In [88]:
grouped = df.groupby(["A"])
grouped.get_group('bar')

,A,B,C,D
1,bar,one,0.477848,0.671811
3,bar,three,-0.005963,0.162831
5,bar,two,-1.771557,0.554359


In [91]:
df.groupby(['A','B']).get_group(('bar', 'one'))

,A,B,C,D
1,bar,one,0.477848,0.671811


In [93]:
grouped = df.groupby("A")
grouped.aggregate(np.sum)

grouped = df.groupby(['A', 'B'])
grouped.aggregate(np.sum)

C         D
A   B                        
bar one    0.477848  0.671811
    three -0.005963  0.162831
    two   -1.771557  0.554359
foo one    2.797174 -1.898627
    three  0.815822  2.583978
    two   -1.028769  0.505831

In [96]:
grouped = df.groupby(["A", "B"], as_index=False)
grouped.aggregate(sum)

df.groupby("A", as_index=False).sum()

,A,C,D
0,bar,-1.299672,1.389001
1,foo,2.584228,1.191182


In [98]:
grouped.size()
grouped.describe()

C                                                                        \
  count      mean       std       min       25%       50%       75%       max   
0   1.0  0.477848       NaN  0.477848  0.477848  0.477848  0.477848  0.477848   
1   1.0 -0.005963       NaN -0.005963 -0.005963 -0.005963 -0.005963 -0.005963   
2   1.0 -1.771557       NaN -1.771557 -1.771557 -1.771557 -1.771557 -1.771557   
3   2.0  1.398587  0.298482  1.187529  1.293058  1.398587  1.504117  1.609646   
4   1.0  0.815822       NaN  0.815822  0.815822  0.815822  0.815822  0.815822   
5   2.0 -0.514384  0.234080 -0.679904 -0.597144 -0.514384 -0.431625 -0.348865   

      D                                                                        
  count      mean       std       min       25%       50%       75%       max  
0   1.0  0.671811       NaN  0.671811  0.671811  0.671811  0.671811  0.671811  
1   1.0  0.162831       NaN  0.162831  0.162831  0.162831  0.162831  0.162831  
2   1.0  0.554359       NaN  0.554359  0.554359  0.554359  0.554359  0.554359  
3   2.0 -0.949313  0.176333 -1.074000 -1.011657 -0.949313 -0.886970 -0.824627  
4   1.0  2.583978       NaN  2.583978  2.583978  2.583978  2.583978  2.583978  
5   2.0  0.252916  0.166328  0.135304  0.194110  0.252916  0.311722  0.370527

In [103]:
ll = [['foo', 1], ['foo', 2], ['foo', 2], ['bar', 1], ['bar', 1]]
df4 = pd.DataFrame(ll, columns=["A", "B"])
print(df4)
print(df4.groupby('A').nunique())
df4.groupby('A').B.value_counts()

     A  B
0  foo  1
1  foo  2
2  foo  2
3  bar  1
4  bar  1
     B
A     
bar  1
foo  2


A    B
bar  1    2
foo  2    2
     1    1
Name: B, dtype: int64

In [105]:
df4.groupby('A').agg(['count', 'size'])

B     
    count size
A             
bar     2    2
foo     3    3

In [108]:
grouped = df.groupby("A")
grouped.C.agg([np.sum, np.mean, np.std])

,sum,mean,std
A,,,
bar,-1.299672,-0.433224,1.184005
foo,2.584228,0.516846,0.989328


In [110]:
grouped.C.agg([np.sum, np.mean, np.std]).rename(columns={'sum':'foo',
                'mean':'bar', 'std':'baz'})

,foo,bar,baz
A,,,
bar,-1.299672,-0.433224,1.184005
foo,2.584228,0.516846,0.989328


In [111]:
animals = pd.DataFrame({
  "kind": ["cat", "dog", "cat", "dog"],
    "height": [9.1, 6.0, 9.5, 34.0],
    "weight": [7.9, 7.5, 9.9, 198.0],
})
animals

,kind,height,weight
0,cat,9.1,7.9
1,dog,6.0,7.5
2,cat,9.5,9.9
3,dog,34.0,198.0


In [112]:
animals.groupby('kind').agg(min_height = ('height', min),
                           max_height = ('height', max),
                           avg_weight = ('weight', 'mean'))

,min_height,max_height,avg_weight
kind,,,
cat,9.1,9.5,8.90
dog,6.0,34.0,102.75


In [115]:
animals.groupby("kind").agg(
    **{
        "total weight": pd.NamedAgg(column="weight", aggfunc=sum)
    })

# Syntactic sugar
animals.groupby("kind").agg(
    **{
        "total weight": ("weight", sum)
    })

,total weight
kind,
cat,17.8
dog,205.5


In [118]:
animals.groupby('kind').height.agg(min_height = min, max_height=max)

,min_height,max_height
kind,,
cat,9.1,9.5
dog,6.0,34.0
